# Quantum Molecular Unfolding
https://arxiv.org/abs/2107.13607

Kevin Mato, Riccardo Mengoni, Daniele Ottaviani, Gianluca Palermo

*Abstract*: Molecular Docking (MD) is an important step of the drug discovery process which aims at calculating the preferred position and shape of one molecule to a second when they are bound to each other. During such analysis, 3D representations of molecules are manipulated according to their degree of freedoms: rigid roto-translation and fragment rotations along the rotatable bonds. In our work, we focused on one specific phase of the molecular docking procedure i.e. Molecular Unfolding (MU), which is used to remove the initial bias of a molecule by expanding it to an unfolded shape. The objective of the MU problem is to find the configuration that maximizes the molecular area, or equivalently, that maximizes the internal distances between atoms inside the molecule. We propose a quantum annealing approach to MU by formulating it as a High-order Unconstrained Binary Optimization (HUBO) which was possible to solve on the latest D-Wave annealing hardware (2000Q and Advantage). Results and performances obtained with quantum annealers are compared with state of art classical solvers.


In [1]:
import sys
sys.path.append('../')
import math
import sympy
from sympy import poly
import numpy
import dimod
from sympy import symbols
from copy import deepcopy
import neal

HUBOs for Quantum Molecular unfolding problem have this form $H(x_{ik})=A\sum_{i} \left( \sum_{k=1}^d x_{ik} -1 \right)^2 - \sum_{\substack{a,b}} D_{ab}( {\Theta})^2$

where $H_A=A\sum_{i} \left( \sum_{k=1}^d x_{ik} -1 \right)^2$ is the hard constraint, modulated by coefficient A which has to be large wrt maximum coefficint appearing in $H_B=\sum_{\substack{a,b}} D_{ab}( {\Theta})^2$ which is the optimization term, responsible for maximization of internal distances.


HUBOs are already generated and are stored in folders. HUBOs are divided in several groups, depending on number of rotatable bonds considered (e.g. 5 rotatable bonds in the folder *./N_Rot_5*) and value of the threshold used to generate the HUBO (e.g. threshold value equal to 100 in folder *./HUBO_5_100*). 

### Select the Molecular HUBO you want to study. 
Check that the path is correct.

In [2]:
HUBO_Mol="./N_Rot_2/HUBO_20A12Rot_27#s2"

Initializing the variables need in the problem

In [3]:
#numer of rotatable bond considered
num_rots=int(HUBO_Mol[HUBO_Mol.find("s")+1])

#Initialize variables
if num_rots==2:
    a0, a1, a2, a3  = symbols('a0, a1, a2, a3')
if num_rots==4:
    a0, a1, a2, a3, a4, a5, a6, a7  = symbols('a0, a1, a2, a3, a4, a5, a6, a7')
if num_rots==5:
    a0, a1, a2, a3, a4, a5, a6, a7, a8, a9  = symbols('a0, a1, a2, a3, a4, a5, a6, a7, a8, a9')
if num_rots==6:
    a0, a1, a2, a3, a4, a5, a6, a7, a8, a9, a10, a11  = symbols('a0, a1, a2, a3, a4, a5, a6, a7, a8, a9, a10, a11')
if num_rots==8:
    a0, a1, a2, a3, a4, a5, a6, a7, a8, a9, a10, a11, a12, a13, a14, a15  = symbols('a0, a1, a2, a3, a4, a5, a6, a7, a8, a9, a10, a11, a12, a13, a14, a15')

Read the correct files in folder.
Files named  *_total*  contain the scoring function $H_B$ to evaluate (in the format of a *sympy polynomial*).

In [4]:
FILE= HUBO_Mol + ".txt"
total_FILE= HUBO_Mol + '_total.txt'

read_total = open(total_FILE, 'r').read()
total=eval(read_total)

Check the max coefficient appearing in $H_B$ and set the hard constraint strenght $A$ as 10 times this number

In [5]:
#This is used to check the maximum coefficient appearing in Hubo_B
A_const=0
read_dictionary_B = open(FILE, 'r').read()
HUBO_B=eval(read_dictionary_B)

#We set the Hard constraint strength as the (maximum coefficient appearing in Hubo_B)*const.
#const was empirically selected to be 10
const=10
A_const=max(map(abs, list(HUBO_B.values())))*const

#read the final HUBO
read_dictionary= open(FILE, 'r').read()
HUBO=eval(read_dictionary)

Current size of the HUBO

In [6]:
print("Current size of the HUBO:",len(HUBO))

Current size of the HUBO: 137


Define the Threshold approximation function.
Coefficints with absolute value less than $10^{val}$ are deleted from the HUBO.

In [9]:
def threshold_approx(h, val=1):
    d =h.copy()
    monoms = h.keys()
    for m in monoms:     
        temp = d[m]
        if (temp < 0.0):
            temp = -1.0 * temp
        if (temp <= (10.0 ** (val))):
            del d[m]
    return d

### Select a threshold value and apply the threshold approximation

In [10]:
#Coefficints with absolute value less than 10^{threshold} are deleted from the HUBO.
threshold=2

HUBO=threshold_approx(HUBO,threshold)

print("Size of the HUBO after threshold approximation:",len(HUBO))

Size of the HUBO after threshold approximation: 73


Constract the binary quadratic model (QUBO) from the HUBO using the function *dimod.make_quadratic* , which takes in input a *strength* parameter set to 1.5*(maximum HUBO value) for empirical reasons (as suggested by D-Wave researchers)

In [11]:
#calculate the strength parameter needed by make_quadratic
max_hubo_value=max(map(abs, list(HUBO.values())))
strength=1.5*max_hubo_value
#generate the bqm
bqm = dimod.make_quadratic(HUBO, strength, dimod.BINARY)

### Select a sampler and the sample_size
Here we consider the Simulated annealing solver. In order to use the Quantum annealing solver, please set up a D-Wave Leap account as explained here  https://docs.ocean.dwavesys.com/en/stable/overview/sapi.html

Inthat case use DWaveSampler() with annealing schedule of $1 \mu second$

*sampler = EmbeddingComposite(DWaveSampler())*

*sampleset = sampler.sample(bqm, num_reads=sample_size, anneal_schedule=[[0.0,0.0],[1.0,1.0]])*

Remember that HUBOs get harder to solve when more rotatable bonds are considerd.
Increase the sample_size accordingly (suggested sample_sizes: 1000, 10000, 100000)


In [12]:
sampler = neal.SimulatedAnnealingSampler()

sample_size=10


Run the problem and display sample

In [13]:
sampleset = sampler.sample(bqm, num_reads=sample_size)
print(sampleset)

   1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16       energy num_oc.
1  0  0  1  0  0  0  0  0  1  0  0  0  0  0  0  0 -2791.285156       1
4  0  0  1  0  0  0  0  0  0  0  0  0  0  0  1  0 -2772.867188       1
2  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  1 -2753.707031       1
3  0  0  0  1  0  0  0  0  0  0  0  0  1  0  0  0 -2736.689209       1
7  1  0  0  0  0  0  0  0  0  0  0  0  0  1  0  0 -2725.957031       1
0  1  0  0  0  0  0  0  0  0  0  0  1  0  0  0  0 -2724.152344       1
8  0  0  0  0  0  1  0  0  1  0  0  0  0  0  0  0 -2722.300781       1
5  0  0  0  0  0  0  0  1  0  0  0  0  0  0  1  0 -2711.679688       1
9  0  0  0  0  0  0  0  1  0  0  0  0  0  0  1  0 -2711.679688       1
6  0  0  0  0  0  0  0  1  0  0  0  0  0  1  0  0 -2697.425781       1
['BINARY', 10 rows, 10 samples, 16 variables]


### PostProcessing Part
Evaluating the sampleset on  $H_B$


In [14]:
new_answer=sampleset.record.sample

counter=0
solutions={}
occur_solutions=[]
occurrences={}

for s in range(len(new_answer)):
    bit_new_answer=new_answer[s]
    
    if num_rots==2:
    
        k1=tuple(i for i,x in enumerate(bit_new_answer[0:8]) if x == 1)
        k2=tuple(i for i,x in enumerate(bit_new_answer[8:16]) if x == 1)
        
        if len(k1)==1:
            if len(k2)==1:
                         
                occur_solutions.append((k1[0]+1, k2[0]+1))
                    
                c1=numpy.cos((numpy.pi/4.0)*(k1[0]+1))
                s1=numpy.sin((numpy.pi/4.0)*(k1[0]+1))
                c2=numpy.cos((numpy.pi/4.0)*(k2[0]+1))
                s2=numpy.sin((numpy.pi/4.0)*(k2[0]+1))
                
                minim=total(c1,s1,c2,s2)
                
                solutions[(k1[0]+1, k2[0]+1)]= minim
                counter=counter+1
                
    if num_rots==4:
        
        k1=tuple(i for i,x in enumerate(bit_new_answer[0:8]) if x == 1)
        k2=tuple(i for i,x in enumerate(bit_new_answer[8:16]) if x == 1)
        k3=tuple(i for i,x in enumerate(bit_new_answer[16:24]) if x == 1)
        k4=tuple(i for i,x in enumerate(bit_new_answer[24:32]) if x == 1)
        
        
        if len(k1)==1:
            if len(k2)==1:
                if len(k3)==1:
                    if len(k4)==1:
                        
                        occur_solutions.append((k1[0]+1, k2[0]+1, k3[0]+1, k4[0]+1))
            
                        c1=numpy.cos((numpy.pi/4.0)*(k1[0]+1))
                        s1=numpy.sin((numpy.pi/4.0)*(k1[0]+1))
                        c2=numpy.cos((numpy.pi/4.0)*(k2[0]+1))
                        s2=numpy.sin((numpy.pi/4.0)*(k2[0]+1))
                        c3=numpy.cos((numpy.pi/4.0)*(k3[0]+1))
                        s3=numpy.sin((numpy.pi/4.0)*(k3[0]+1))
                        c4=numpy.cos((numpy.pi/4.0)*(k4[0]+1))
                        s4=numpy.sin((numpy.pi/4.0)*(k4[0]+1))
                        
                        minim=total(c1,s1,c2,s2,c3,s3,c4,s4)
               
                        solutions[(k1[0]+1, k2[0]+1, k3[0]+1, k4[0]+1)]= minim
                        counter=counter+1
                        
    if num_rots==5:
        
        k1=tuple(i for i,x in enumerate(bit_new_answer[0:8]) if x == 1)
        k2=tuple(i for i,x in enumerate(bit_new_answer[8:16]) if x == 1)
        k3=tuple(i for i,x in enumerate(bit_new_answer[16:24]) if x == 1)
        k4=tuple(i for i,x in enumerate(bit_new_answer[24:32]) if x == 1)
        k5=tuple(i for i,x in enumerate(bit_new_answer[32:40]) if x == 1)
    
        
        if len(k1)==1:
            if len(k2)==1:
                if len(k3)==1:
                    if len(k4)==1:
                        if len(k5)==1:
                            
                            occur_solutions.append((k1[0]+1, k2[0]+1, k3[0]+1, k4[0]+1, k5[0]+1))
                            
                            c1=numpy.cos((numpy.pi/4.0)*(k1[0]+1))
                            s1=numpy.sin((numpy.pi/4.0)*(k1[0]+1))
                            c2=numpy.cos((numpy.pi/4.0)*(k2[0]+1))
                            s2=numpy.sin((numpy.pi/4.0)*(k2[0]+1))
                            c3=numpy.cos((numpy.pi/4.0)*(k3[0]+1))
                            s3=numpy.sin((numpy.pi/4.0)*(k3[0]+1))
                            c4=numpy.cos((numpy.pi/4.0)*(k4[0]+1))
                            s4=numpy.sin((numpy.pi/4.0)*(k4[0]+1))
                            c5=numpy.cos((numpy.pi/4.0)*(k5[0]+1))
                            s5=numpy.sin((numpy.pi/4.0)*(k5[0]+1))

                            minim=total(c1,s1,c2,s2,c3,s3,c4,s4,c5,s5)
                  
                            solutions[(k1[0]+1, k2[0]+1, k3[0]+1, k4[0]+1, k5[0]+1)]= minim
                            counter=counter+1
                            
    if num_rots==6:
        
        k1=tuple(i for i,x in enumerate(bit_new_answer[0:8]) if x == 1)
        k2=tuple(i for i,x in enumerate(bit_new_answer[8:16]) if x == 1)
        k3=tuple(i for i,x in enumerate(bit_new_answer[16:24]) if x == 1)
        k4=tuple(i for i,x in enumerate(bit_new_answer[24:32]) if x == 1)
        k5=tuple(i for i,x in enumerate(bit_new_answer[32:40]) if x == 1)
        k6=tuple(i for i,x in enumerate(bit_new_answer[40:48]) if x == 1)
        
        
        if len(k1)==1:
            if len(k2)==1:
                if len(k3)==1:
                    if len(k4)==1:
                        if len(k5)==1:
                            if len(k6)==1:
                        
                                occur_solutions.append((k1[0]+1, k2[0]+1, k3[0]+1, k4[0]+1, k5[0]+1, k6[0]+1))
            
                                c1=numpy.cos((numpy.pi/4.0)*(k1[0]+1))
                                s1=numpy.sin((numpy.pi/4.0)*(k1[0]+1))
                                c2=numpy.cos((numpy.pi/4.0)*(k2[0]+1))
                                s2=numpy.sin((numpy.pi/4.0)*(k2[0]+1))
                                c3=numpy.cos((numpy.pi/4.0)*(k3[0]+1))
                                s3=numpy.sin((numpy.pi/4.0)*(k3[0]+1))
                                c4=numpy.cos((numpy.pi/4.0)*(k4[0]+1))
                                s4=numpy.sin((numpy.pi/4.0)*(k4[0]+1))
                                c5=numpy.cos((numpy.pi/4.0)*(k5[0]+1))
                                s5=numpy.sin((numpy.pi/4.0)*(k5[0]+1))
                                c6=numpy.cos((numpy.pi/4.0)*(k6[0]+1))
                                s6=numpy.sin((numpy.pi/4.0)*(k6[0]+1))
                                
                                minim=total(c1,s1,c2,s2,c3,s3,c4,s4,c5,s5,c6,s6)
                      
                                solutions[(k1[0]+1, k2[0]+1, k3[0]+1, k4[0]+1, k5[0]+1, k6[0]+1)]= minim
                                counter=counter+1
                                
    if num_rots==8:
        
        k1=tuple(i for i,x in enumerate(bit_new_answer[0:8]) if x == 1)
        k2=tuple(i for i,x in enumerate(bit_new_answer[8:16]) if x == 1)
        k3=tuple(i for i,x in enumerate(bit_new_answer[16:24]) if x == 1)
        k4=tuple(i for i,x in enumerate(bit_new_answer[24:32]) if x == 1)
        k5=tuple(i for i,x in enumerate(bit_new_answer[32:40]) if x == 1)
        k6=tuple(i for i,x in enumerate(bit_new_answer[40:48]) if x == 1)
        k7=tuple(i for i,x in enumerate(bit_new_answer[48:56]) if x == 1)
        k8=tuple(i for i,x in enumerate(bit_new_answer[56:64]) if x == 1)
        
        
        
        if len(k1)==1:
            if len(k2)==1:
                if len(k3)==1:
                    if len(k4)==1:
                        if len(k5)==1:
                            if len(k6)==1:
                                if len(k7)==1:
                                    if len(k8)==1:
                        
                                        occur_solutions.append((k1[0]+1, k2[0]+1, k3[0]+1, k4[0]+1, k5[0]+1, k6[0]+1, k7[0]+1, k8[0]+1))
                    
                                        c1=numpy.cos((numpy.pi/4.0)*(k1[0]+1))
                                        s1=numpy.sin((numpy.pi/4.0)*(k1[0]+1))
                                        c2=numpy.cos((numpy.pi/4.0)*(k2[0]+1))
                                        s2=numpy.sin((numpy.pi/4.0)*(k2[0]+1))
                                        c3=numpy.cos((numpy.pi/4.0)*(k3[0]+1))
                                        s3=numpy.sin((numpy.pi/4.0)*(k3[0]+1))
                                        c4=numpy.cos((numpy.pi/4.0)*(k4[0]+1))
                                        s4=numpy.sin((numpy.pi/4.0)*(k4[0]+1))
                                        c5=numpy.cos((numpy.pi/4.0)*(k5[0]+1))
                                        s5=numpy.sin((numpy.pi/4.0)*(k5[0]+1))
                                        c6=numpy.cos((numpy.pi/4.0)*(k6[0]+1))
                                        s6=numpy.sin((numpy.pi/4.0)*(k6[0]+1))
                                        c7=numpy.cos((numpy.pi/4.0)*(k7[0]+1))
                                        s7=numpy.sin((numpy.pi/4.0)*(k7[0]+1))
                                        c8=numpy.cos((numpy.pi/4.0)*(k8[0]+1))
                                        s8=numpy.sin((numpy.pi/4.0)*(k8[0]+1))
                                        
                                        minim=total(c1,s1,c2,s2,c3,s3,c4,s4,c5,s5,c6,s6,c7,s7,c8,s8)
                                                                  
                                        solutions[(k1[0]+1, k2[0]+1, k3[0]+1, k4[0]+1, k5[0]+1, k6[0]+1, k7[0]+1, k8[0]+1)]= minim
    
                                        counter=counter+1
    

for sols in  list(set(occur_solutions)):
    occurrences[sols]=occur_solutions.count(sols)
    
Final_Solutions= sorted(solutions.items(), key=lambda x: x[1], reverse=True)

## Display Solutions

Number of acceptable solutions that do not violate the hard contraint:

In [15]:
print( "Number of acceptable solutions found: ", counter)

Number of acceptable solutions found:  10


Dictionary containing all the acceptable solutions found. Each solution is of the following form
$[(k_1..k_d), H_B(k_1..k_d)]$ where $(k_1..k_d)$ are the parameters that define the torsional angle value $\theta_i=\frac{\pi}{4}k_i$, while $H_B(k_1..k_d)$ is the value of the optimization function, evaluated in $(k_1..k_d)$

In [16]:
print("All solutions found: ", Final_Solutions)

All solutions found:  [((3, 1), 220.704690075685), ((1, 8), 215.589769852800), ((3, 7), 211.100698922578), ((4, 5), 196.025016740852), ((1, 6), 176.921160968135), ((8, 7), 171.939415235912), ((6, 1), 170.560071322753), ((1, 4), 149.811102703967), ((8, 6), 141.934996918969)]


Occurrences of each solution $(k_1..k_d)$

In [17]:
print( "Occurrences of each solution: ", occurrences)

Occurrences of each solution:  {(3, 1): 1, (3, 7): 1, (8, 7): 2, (1, 8): 1, (6, 1): 1, (1, 4): 1, (4, 5): 1, (8, 6): 1, (1, 6): 1}


Best solution found  $(k_1..k_d)_{best}$ and its occurrency 

In [18]:
print( "Best Solution Found: ", Final_Solutions[0])
print( "Best Solution Occurences: ", occurrences[Final_Solutions[0][0]])

Best Solution Found:  ((3, 1), 220.704690075685)
Best Solution Occurences:  1
